In [1]:
# 기본 패키지
import sqlite3
import requests #배부 source를 가져오는 라이브러리
from bs4 import BeautifulSoup #
import numpy as np
import pandas as pd

In [2]:
# 함수화 
# 반복사용할 부분 블럭화
url="https://finance.naver.com/item/sise_day.nhn?code=069960&page=1"
def stockGet(url):
    src=requests.get(url)
    result=src.content.strip()
    html=BeautifulSoup(result, 'html.parser')
    tgts=html.select('table.type2 tr')
    res=[]

    for tgt in tgts:
        if (tgt.select('td .p10')):
            sday=tgt.select('td .p10')[0].text.replace(".", "")
        
            endp=tgt.select('td.num')[0].text.replace(",", "")
            startp=tgt.select('td.num')[2].text.replace(",", "")
            maxp=tgt.select('td.num')[3].text.replace(",", "")
            minp=tgt.select('td.num')[4].text.replace(",", "")
            vol=tgt.select('td.num')[5].text.replace(",", "")
            #종가/전일비/시가/고가/저가
            res.append((sday, endp, startp, maxp, minp, endp, vol))
    return res

In [3]:
url="https://finance.naver.com/item/sise_day.nhn?code=069960&page=1"
stockGet(url)

[('20200915', '57400', '57500', '58000', '56600', '57400', '219218'),
 ('20200914', '57200', '57000', '58400', '56700', '57200', '324102'),
 ('20200911', '56300', '56400', '57400', '56200', '56300', '269437'),
 ('20200910', '56400', '56500', '57200', '55900', '56400', '262036'),
 ('20200909', '56000', '55900', '56600', '55400', '56000', '125147'),
 ('20200908', '56400', '56800', '57100', '56300', '56400', '152605'),
 ('20200907', '56300', '56100', '57100', '55600', '56300', '174472'),
 ('20200904', '55800', '55300', '56200', '55100', '55800', '194702'),
 ('20200903', '56500', '57000', '58000', '56500', '56500', '287154'),
 ('20200902', '56500', '56100', '57800', '55300', '56500', '370906')]

In [4]:
dbname='NaverStock.db'
def addData(data):
    with sqlite3.connect(dbname) as conn:
        cur=conn.cursor()
        sql = 'insert into StockDay \
        (SIDX,SCODE,SCOMP,SDAY,OPENP,HIGHP,LOWP,CLOSEP,VOL) \
        values (?,?,?,?,?,?,?,?,?)'
        ##### data=[1001,60000,' 신세계','20200830',30,60,70,50,250]
        cur.execute(sql,data)
        conn.commit()
        cur.close()

In [5]:
dbname='NaverStock.db'
def selData(whr=' where SIDX=0'):
    with sqlite3.connect(dbname) as conn:
        cur=conn.cursor()
        sql='select SIDX from StockDay '+whr
        cur.execute(sql)
        rows=cur.fetchall()
        cur.close()
    return rows

In [6]:
urlsrc="https://finance.naver.com/item/sise_day.nhn?code={0}&page={1}"
sCode='069960';
sComp='현대백화점'
for i in range(1,3):
    print('#'*30)
    url=urlsrc.format(sCode,i)
    print(url)
    rs=stockGet(url)
    for r in rs:
        sidx=r[0]+sCode
        data=[sidx, sCode, sComp, r[0],r[1],r[2],r[3],r[4],r[5]]
        selData('where SIDX='+sidx)
        preData=selData(' where SIDX='+sidx)
        if(preData):
            print("중복데이터 입니다", sidx)
        else:
            addData(data)

##############################
https://finance.naver.com/item/sise_day.nhn?code=069960&page=1
##############################
https://finance.naver.com/item/sise_day.nhn?code=069960&page=2


In [12]:
def MyCrawl(comp=('현대백화점','069960'),sp=1,ep=2):
    sCode=comp[1]
    sComp=comp[0] #~441
    urlsrc='https://finance.naver.com/item/sise_day.nhn?code={0}&page={1}'
    for i in range(1,5):
        print("#"*30)
        url=urlsrc.format(sCode,i)
        print(url)
        # 데이터 10개 가져오기
        rs=stockGet(url)

        for r in rs:
            #데이터별 고유코드만들기
            sidx=r[0]+sCode
            ##### data=[1001,60000,' 신세계','20200830',30,60,70,50,250]
            data=[sidx,sCode,sComp,r[0],r[1],r[2],r[3],r[4],r[5]]
            #데이터 입력하기 중복데이터 확인
            preData=selData(' where SIDX='+sidx)
            if(preData):
                print("중복데이터 입니다.",preData)
            else:
                #print(r)
                addData(data)

In [13]:
complist=[("현대백화점","069960"),("영원무역","111770"),\
         ("SK네트웍스","001740"),("신세계인터네셔널","031430")]
MyCrawl(complist[0],1,2)

##############################
https://finance.naver.com/item/sise_day.nhn?code=069960&page=1
중복데이터 입니다. [(20200915069960,)]
중복데이터 입니다. [(20200914069960,)]
중복데이터 입니다. [(20200911069960,)]
중복데이터 입니다. [(20200910069960,)]
중복데이터 입니다. [(20200909069960,)]
중복데이터 입니다. [(20200908069960,)]
중복데이터 입니다. [(20200907069960,)]
중복데이터 입니다. [(20200904069960,)]
중복데이터 입니다. [(20200903069960,)]
중복데이터 입니다. [(20200902069960,)]
##############################
https://finance.naver.com/item/sise_day.nhn?code=069960&page=2
중복데이터 입니다. [(20200901069960,)]
중복데이터 입니다. [(20200831069960,)]
중복데이터 입니다. [(20200828069960,)]
중복데이터 입니다. [(20200827069960,)]
중복데이터 입니다. [(20200826069960,)]
중복데이터 입니다. [(20200825069960,)]
중복데이터 입니다. [(20200824069960,)]
중복데이터 입니다. [(20200821069960,)]
중복데이터 입니다. [(20200820069960,)]
중복데이터 입니다. [(20200819069960,)]
##############################
https://finance.naver.com/item/sise_day.nhn?code=069960&page=3
##############################
https://finance.naver.com/item/sise_day.nhn?code=069960&page=4


In [10]:
MyCrawl(complist[1],1,2)

##############################
https://finance.naver.com/item/sise_day.nhn?code=111770&page=1


In [15]:
def MyCrawl(comp=('현대백화점','069960'),sp=1,ep=2):
    sCode=comp[1]
    sComp=comp[0] #~441
    urlsrc='https://finance.naver.com/item/sise_day.nhn?code={0}&page={1}'
    for i in range(sp,ep):
        print("#"*30)
        url=urlsrc.format(sCode,i)
        print(url)
        # 데이터 10개 가져오기
        rs=stockGet(url)

        for r in rs:
            #데이터별 고유코드만들기
            sidx=r[0]+sCode
            ##### data=[1001,60000,' 신세계','20200830',30,60,70,50,250]
            data=[sidx,sCode,sComp,r[0],r[1],r[2],r[3],r[4],r[5]]
            #데이터 입력하기 중복데이터 확인
            preData=selData(' where SIDX='+sidx)
            if(preData):
                print("중복데이터 입니다.",preData)
            else:
                #print(r)
                addData(data)

In [17]:
MyCrawl(complist[0],1,8)

##############################
https://finance.naver.com/item/sise_day.nhn?code=069960&page=1
중복데이터 입니다. [(20200915069960,)]
중복데이터 입니다. [(20200914069960,)]
중복데이터 입니다. [(20200911069960,)]
중복데이터 입니다. [(20200910069960,)]
중복데이터 입니다. [(20200909069960,)]
중복데이터 입니다. [(20200908069960,)]
중복데이터 입니다. [(20200907069960,)]
중복데이터 입니다. [(20200904069960,)]
중복데이터 입니다. [(20200903069960,)]
중복데이터 입니다. [(20200902069960,)]
##############################
https://finance.naver.com/item/sise_day.nhn?code=069960&page=2
중복데이터 입니다. [(20200901069960,)]
중복데이터 입니다. [(20200831069960,)]
중복데이터 입니다. [(20200828069960,)]
중복데이터 입니다. [(20200827069960,)]
중복데이터 입니다. [(20200826069960,)]
중복데이터 입니다. [(20200825069960,)]
중복데이터 입니다. [(20200824069960,)]
중복데이터 입니다. [(20200821069960,)]
중복데이터 입니다. [(20200820069960,)]
중복데이터 입니다. [(20200819069960,)]
##############################
https://finance.naver.com/item/sise_day.nhn?code=069960&page=3
중복데이터 입니다. [(20200818069960,)]
중복데이터 입니다. [(20200814069960,)]
중복데이터 입니다. [(20200813069960,)]
중복데이터

In [18]:
MyCrawl(complist[1],1,8)

##############################
https://finance.naver.com/item/sise_day.nhn?code=111770&page=1
중복데이터 입니다. [(20200915111770,)]
중복데이터 입니다. [(20200914111770,)]
중복데이터 입니다. [(20200911111770,)]
중복데이터 입니다. [(20200910111770,)]
중복데이터 입니다. [(20200909111770,)]
중복데이터 입니다. [(20200908111770,)]
중복데이터 입니다. [(20200907111770,)]
중복데이터 입니다. [(20200904111770,)]
중복데이터 입니다. [(20200903111770,)]
중복데이터 입니다. [(20200902111770,)]
##############################
https://finance.naver.com/item/sise_day.nhn?code=111770&page=2
##############################
https://finance.naver.com/item/sise_day.nhn?code=111770&page=3
##############################
https://finance.naver.com/item/sise_day.nhn?code=111770&page=4
##############################
https://finance.naver.com/item/sise_day.nhn?code=111770&page=5
##############################
https://finance.naver.com/item/sise_day.nhn?code=111770&page=6
##############################
https://finance.naver.com/item/sise_day.nhn?code=111770&page=7


In [19]:
MyCrawl(complist[2],1,8)

##############################
https://finance.naver.com/item/sise_day.nhn?code=001740&page=1
##############################
https://finance.naver.com/item/sise_day.nhn?code=001740&page=2
##############################
https://finance.naver.com/item/sise_day.nhn?code=001740&page=3
##############################
https://finance.naver.com/item/sise_day.nhn?code=001740&page=4
##############################
https://finance.naver.com/item/sise_day.nhn?code=001740&page=5
##############################
https://finance.naver.com/item/sise_day.nhn?code=001740&page=6
##############################
https://finance.naver.com/item/sise_day.nhn?code=001740&page=7


In [20]:
MyCrawl(complist[3],1,8)

##############################
https://finance.naver.com/item/sise_day.nhn?code=031430&page=1
##############################
https://finance.naver.com/item/sise_day.nhn?code=031430&page=2
##############################
https://finance.naver.com/item/sise_day.nhn?code=031430&page=3
##############################
https://finance.naver.com/item/sise_day.nhn?code=031430&page=4
##############################
https://finance.naver.com/item/sise_day.nhn?code=031430&page=5
##############################
https://finance.naver.com/item/sise_day.nhn?code=031430&page=6
##############################
https://finance.naver.com/item/sise_day.nhn?code=031430&page=7
